<a href="https://colab.research.google.com/github/alfazick/GenerativeAI/blob/main/SpaCyRule_Based_Matchmaking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:


!python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 587.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# quickly extract information from text by matching patterns and phrases
# use morphological features, POS tags, regex
# to form pattern objects to feed to the Matcher objects

In [3]:
# Token-based matching
# PhraseMatcher
# EntityRuler
# Combining spaCy modesl and matchers


In [1]:
# 1 Token Based Matching
# Matching rules can refer to the token or it's linguistic attributes

import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [11]:
doc = nlp("Good MORnIng, I want to reserve a ticket.")
matcher = Matcher(nlp.vocab)
pattern = [{"LOWER":"good"},{"LOWER":"morning"},{"IS_PUNCT":True}]
matcher.add("morningGreeting",[pattern])
matches = matcher(doc)
for match_id, start, end in matches:
    m_span = doc[start:end]
    print(start,end, m_span.text)

0 3 Good MORnIng,


In [5]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

def address_extractor(text,address_pattern,nlp_model):
    matcher = Matcher(nlp_model.vocab)
    # Add the address pattern to the matcher
    matcher.add("ADDRESS", [address_pattern])
    doc = nlp_model(text)
    matches = matcher(doc)
    # Return both the matches and the doc to extract text outside
    return matches, doc


# Define a pattern for a basic address: arbitrary digit number, street name, street type, optional zip code
address_pattern = [
    {"SHAPE": "d+"},  # Street number: one or more digits
    {"IS_ALPHA": True, "LENGTH": {">=": 2}},  # Street name: two or more alphabetical characters
    {"LOWER": {"IN": ["st", "street", "ave", "avenue", "blvd", "boulevard", "road", "rd"]}},  # Street type
    {"OP": "?"},  # Make the previous token optional
    {"SHAPE": "dddd", "OP": "?"},  # Optional zip code: 5 digits
    {"TEXT": "-", "OP": "?"},  # Optional hyphen for extended zip code format
    {"SHAPE": "dddd", "OP": "?"}  # Optional extension of the zip code
]

address_pattern = [
    {"SHAPE": "d+"},  # Street number: one or more digits
    {"IS_ALPHA": True, "OP": "*", "LENGTH": {">=": 1}},  # Optional street name: one or more alphabetical characters
    {"IS_PUNCT": True, "OP": "?"},  # Optional punctuation
    {"LOWER": {"IN": ["st", "street", "ave", "avenue", "blvd", "boulevard", "road", "rd", "ln", "lane", "dr", "drive"]}, "OP": "?"},  # Optional street type
    {"IS_PUNCT": True, "OP": "?"},  # Optional punctuation
    {"SHAPE": {"REGEX": "^(\\d{5}(-\\d{4})?|[A-Z]{1,2}\\d[A-Z\\d]? \\d[A-Z]{2})$"}, "OP": "?"}  # Optional zip code or postcode, supporting US and UK formats
]
address_pattern = [
        {"SHAPE": "d+"},  # Street number
        {"IS_ALPHA": True, "OP": "?"},  # Optional street direction (N, S, E, W)
        {"IS_ALPHA": True},  # Street name
        {"LOWER": {"IN": ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "drive", "dr"]}},  # Street type
    ]

pseudo_addresses = [
    "123 Main St, Springfield, IL 62704",
    "456 Elm Street, Apt 101, New York, NY 10025",
    "789 Pine Avenue, Los Angeles, CA 90015",
    "Unit 5, 250 Cherry Blvd, Seattle, WA 98104",
    "12B High Tower, Gotham City, 99999",
    "42 Douglas Adams Road, Sector ZZ9 Plural Z Alpha, 4242",
    "88 Rising Sun Lane, Kyoto, 600-8216",
    "1600 Pennsylvania Ave NW, Washington, DC 20500",
    "221B Baker Street, London, NW1 6XE",
    "Flat 4, 28 Marina Bay, Singapore, 018981",
    "Let's meet at 123 Main St. and then go to 456 Elm Avenue for lunch."
]

texts = pseudo_addresses[:]

# Testing
for text in texts:
    print(f"Original Text: {text}")
    matches, doc = address_extractor(text, address_pattern, nlp)  # Adjusted to receive both matches and doc
    for match_id, start, end in matches:
        match_span = doc[start:end]  # Extracting match text here
        print(f"Match found: {match_span.text} (Start: {start}, End: {end})")
    if not matches:
        print("No matches found.")

Original Text: 123 Main St, Springfield, IL 62704
No matches found.
Original Text: 456 Elm Street, Apt 101, New York, NY 10025
No matches found.
Original Text: 789 Pine Avenue, Los Angeles, CA 90015
No matches found.
Original Text: Unit 5, 250 Cherry Blvd, Seattle, WA 98104
No matches found.
Original Text: 12B High Tower, Gotham City, 99999
No matches found.
Original Text: 42 Douglas Adams Road, Sector ZZ9 Plural Z Alpha, 4242
No matches found.
Original Text: 88 Rising Sun Lane, Kyoto, 600-8216
No matches found.
Original Text: 1600 Pennsylvania Ave NW, Washington, DC 20500
No matches found.
Original Text: 221B Baker Street, London, NW1 6XE
No matches found.
Original Text: Flat 4, 28 Marina Bay, Singapore, 018981
No matches found.
Original Text: Let's meet at 123 Main St. and then go to 456 Elm Avenue for lunch.
No matches found.


In [8]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_lg")
matcher = Matcher(nlp.vocab)

# Define pattern for US-style addresses
address_pattern = [
    {"SHAPE": "d+"},  # Street number: one or more digits
    {"IS_ALPHA": True, "LENGTH": {">=": 2}},  # Street name: two or more alphabetical characters
    {"LOWER": {"IN": ["st", "street", "ave", "avenue", "blvd", "boulevard", "road", "rd"]}},  # Street type
    {"OP": "?"},  # Make the previous token optional
    {"SHAPE": "dddd", "OP": "?"},  # Optional zip code: 5 digits
    {"TEXT": "-", "OP": "?"},  # Optional hyphen for extended zip code format
    {"SHAPE": "dddd", "OP": "?"}  # Optional extension of the zip code
]

matcher.add("US_ADDRESS", [address_pattern])

text = "Let's meet at 123 Main St. and then go to 456 Elm Avenue for lunch."
doc = nlp(text)
matches = matcher(doc)

addresses = []
for match_id, start, end in matches:
    span = doc[start:end]  # The matched span
    addresses.append(span.text)
print(addresses)


[]


In [12]:
import spacy
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")  # Using a small model for simplicity

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Define the pattern for street types
street_type_pattern = [
    {"LOWER": {"IN": ["st", "street", "str", "ave", "avenue", "rd", "road", "blvd", "boulevard", "lane", "ln", "dr", "drive"]}}
]

# Add the pattern to the matcher
matcher.add("STREET_TYPE", [street_type_pattern])

def find_street_types(text):
    # Process the text
    doc = nlp(text)
    # Find matches in the text
    matches = matcher(doc)

    # Extract and return the matched street types
    return [doc[start:end].text for match_id, start, end in matches]

# Example text
text = "I've lived on Elm Street, Oak Rd, and Pine Avenue."

# Find and print street types
street_types = find_street_types(text)
print(street_types)


['Street', 'Rd', 'Avenue']


In [15]:
import spacy
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")  # Using a small model for simplicity

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Define the pattern to include the preceding word (street name) and the street type
street_type_pattern = [
    {"IS_ALPHA": True, "OP": "?"},  # Optionally match an adjective (e.g., "Old" in "Old Main Street")
    {"IS_ALPHA": True},  # The word immediately preceding the street type (street name)
    {"LOWER": {"IN": ["st.","st", "street", "str", "ave", "avenue", "rd", "road", "blvd", "boulevard", "lane", "ln", "dr", "drive"]}}  # Street type
]

# Add the pattern to the matcher
matcher.add("STREET_NAME_AND_TYPE", [street_type_pattern])

def find_street_names_and_types(text):
    # Process the text with spaCy
    doc = nlp(text)
    # Find matches in the text
    matches = matcher(doc)

    # Extract and return the matched spans (street names and types)
    return [doc[start:end].text for match_id, start, end in matches]

pseudo_addresses = [
    "123 Main St, Springfield, IL 62704",
    "456 Elm Street, Apt 101, New York, NY 10025",
    "789 Pine Avenue, Los Angeles, CA 90015",
    "Unit 5, 250 Cherry Blvd, Seattle, WA 98104",
    "12B High Tower, Gotham City, 99999",
    "42 Douglas Adams Road, Sector ZZ9 Plural Z Alpha, 4242",
    "88 Rising Sun Lane, Kyoto, 600-8216",
    "1600 Pennsylvania Ave NW, Washington, DC 20500",
    "221B Baker Street, London, NW1 6XE",
    "Flat 4, 28 Marina Bay, Singapore, 018981",
    "Let's meet at 123 Main St. and then go to 456 Elm Avenue for lunch."
]

texts = pseudo_addresses[:]
for text in texts:

    # Find and print street names along with their types
    street_names_and_types = find_street_names_and_types(text)
    print(street_names_and_types)


['Main St']
['Elm Street']
['Pine Avenue']
['Cherry Blvd']
[]
['Douglas Adams Road', 'Adams Road']
['Rising Sun Lane', 'Sun Lane']
['Pennsylvania Ave']
['Baker Street']
[]
['Main St.', 'Elm Avenue']


In [16]:
# Adding two patterns
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
doc = nlp("Good morning, I want to reserve a ticket. I will then say good evening!")
matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": "good"}, {"LOWER": "morning"}, {"IS_PUNCT": True}]
matcher.add("morningGreeting", [pattern])

pattern2 = [{"LOWER": "good"}, {"LOWER": "evening"}, {"IS_PUNCT": True}]
matcher.add("eveningGreeting", [pattern2])
matches = matcher(doc)
for match_id, start, end in matches:
    pattern_name = nlp.vocab.strings[match_id]
    m_span = doc[start:end]
    print(start, end, m_span.text)

0 3 Good morning,
14 17 good evening!


In [18]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
doc = nlp("123 Main St, Springfield, IL 62704",)
matcher = Matcher(nlp.vocab)
pattern = [{"LENGTH": 2}]
matcher.add("onlyShort", [pattern])
matches = matcher(doc)
for match_id, start, end in matches:
    m_span = doc[start:end]
    print(start, end, m_span.text)

# Extract states?


2 3 St
6 7 IL


In [19]:
import spacy
from spacy.matcher import Matcher

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

# Define the pattern to include the preceding word (street name) and the street type
street_type_pattern = [
    {"IS_ALPHA": True, "OP": "?"},  # Optionally match an adjective (e.g., "Old" in "Old Main Street")
    {"IS_ALPHA": True},  # The word immediately preceding the street type (street name)
    {"LOWER": {"IN": ["st.", "st", "street", "str", "ave", "avenue", "rd", "road", "blvd", "boulevard", "lane", "ln", "dr", "drive"]}},  # Street type
]

# Pattern for matching U.S. state abbreviations
state_abbr_pattern = [
    {"LENGTH": 2, "IS_UPPER": True}
]

# Add the patterns to the matcher
matcher.add("STREET_NAME_AND_TYPE", [street_type_pattern])
matcher.add("STATE_ABBR", [state_abbr_pattern])

def find_street_names_types_states(text):
    # Process the text with spaCy
    doc = nlp(text)
    # Find matches in the text
    matches = matcher(doc)

    streets = []
    states = []

    for match_id, start, end in matches:
        span = doc[start:end]  # The matched span
        rule_id = nlp.vocab.strings[match_id]  # Get the pattern name
        if rule_id == "STREET_NAME_AND_TYPE":
            streets.append(span.text)
        elif rule_id == "STATE_ABBR":
            states.append(span.text)

    return streets, states

pseudo_addresses = [
    "123 Main St, Springfield, IL 62704",
    "456 Elm Street, Apt 101, New York, NY 10025",
    "789 Pine Avenue, Los Angeles, CA 90015",
    "Unit 5, 250 Cherry Blvd, Seattle, WA 98104",
    "12B High Tower, Gotham City, 99999",
    "42 Douglas Adams Road, Sector ZZ9 Plural Z Alpha, 4242",
    "88 Rising Sun Lane, Kyoto, 600-8216",
    "1600 Pennsylvania Ave NW, Washington, DC 20500",
    "221B Baker Street, London, NW1 6XE",
    "Flat 4, 28 Marina Bay, Singapore, 018981",
    "Let's meet at 123 Main St. and then go to 456 Elm Avenue for lunch."
]

for text in pseudo_addresses:
    street_names_and_types, state_abbrs = find_street_names_types_states(text)
    print(f"Text: {text}")
    print(f"Street Names and Types: {street_names_and_types}")
    print(f"State Abbreviations: {state_abbrs}")
    print("---")


Text: 123 Main St, Springfield, IL 62704
Street Names and Types: ['Main St']
State Abbreviations: ['IL']
---
Text: 456 Elm Street, Apt 101, New York, NY 10025
Street Names and Types: ['Elm Street']
State Abbreviations: ['NY']
---
Text: 789 Pine Avenue, Los Angeles, CA 90015
Street Names and Types: ['Pine Avenue']
State Abbreviations: ['CA']
---
Text: Unit 5, 250 Cherry Blvd, Seattle, WA 98104
Street Names and Types: ['Cherry Blvd']
State Abbreviations: ['WA']
---
Text: 12B High Tower, Gotham City, 99999
Street Names and Types: []
State Abbreviations: []
---
Text: 42 Douglas Adams Road, Sector ZZ9 Plural Z Alpha, 4242
Street Names and Types: ['Douglas Adams Road', 'Adams Road']
State Abbreviations: []
---
Text: 88 Rising Sun Lane, Kyoto, 600-8216
Street Names and Types: ['Rising Sun Lane', 'Sun Lane']
State Abbreviations: []
---
Text: 1600 Pennsylvania Ave NW, Washington, DC 20500
Street Names and Types: ['Pennsylvania Ave']
State Abbreviations: ['NW', 'DC']
---
Text: 221B Baker Street,

In [22]:
# Observation: Funneling Approach:To consider
# 1) Step extract states possibly?
# 2) Work with text that possibly contain address and extract them

import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans  # Utility to handle overlapping spans

nlp = spacy.load("en_core_web_sm")

matcher = Matcher(nlp.vocab)

# Define the pattern to include the preceding word (street name) and the street type
street_type_pattern = [
    {"IS_ALPHA": True, "OP": "?"},  # Optionally match an adjective (e.g., "Old" in "Old Main Street")
    {"IS_ALPHA": True},  # The word immediately preceding the street type (street name)
    {"LOWER": {"IN": ["st.", "st", "street", "str", "ave", "avenue", "rd", "road", "blvd", "boulevard", "lane", "ln", "dr", "drive"]}},  # Street type
]

# Pattern for matching U.S. state abbreviations
state_abbr_pattern = [
    {"LENGTH": 2, "IS_UPPER": True, "TEXT": {"REGEX": "^(AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY)$"}}
]

# Add the patterns to the matcher
matcher.add("STREET_NAME_AND_TYPE", [street_type_pattern])
matcher.add("STATE_ABBR", [state_abbr_pattern])

def extract_relevant_sentences(text):
    doc = nlp(text)
    relevant_sentences = set()

    for sent in doc.sents:
        if any(matcher(nlp(sent.text))):
            relevant_sentences.add(sent.text)

    return list(relevant_sentences)

def extract_addresses_from_sentences(sentences):
    extracted_addresses = []
    for sentence in sentences:
        doc = nlp(sentence)
        matches = matcher(doc)
        addresses = [doc[start:end].text for match_id, start, end in matches if nlp.vocab.strings[match_id] == "STREET_NAME_AND_TYPE"]
        extracted_addresses.extend(addresses)
    return extracted_addresses

# Example long text
long_text = """
This weekend promises to be an adventure. We're kicking things off with a morning hike in the local park,
where the trails wind through ancient forests and open up to breathtaking views of the valley.
Afterward, a quick visit to the farmer's market downtown is a must, where the freshest produce and homemade goodies await.

Lunch is planned at a small, hidden gem of a cafe known for its farm-to-table approach and quirky decor.
It's located right off the main plaza, but without a sign, it's a place you'd only know if you were a local.
The afternoon will be spent at the museum on 5th, showcasing a new exhibit on the golden age of science fiction.

Dinner reservations are at 7 PM at 'The Orchard', a cozy restaurant nestled within the city's
historical district at 456 Elm Street, Apt 101, New York, NY.
Known for its innovative cuisine and award-winning chef, it's the perfect spot to celebrate our journey's beginning.

The night doesn't end there. We've tickets to a late show at the Grand Theater,
renowned for its opulent design and acoustics that make every performance unforgettable.
As the curtain falls, our thoughts will already be on tomorrow's plans, which include a visit
to the famous 789 Pine Avenue, Los Angeles, CA, for a guided tour of historic homes and gardens.

Sunday is left deliberately open, a canvas for spontaneity. Perhaps a day at the beach,
or exploring the city's array of boutique shops and art galleries.
As the sun sets on our weekend adventure, we'll gather for drinks at a rooftop
bar overlooking the skyline, reflecting on the memories made and the tales to tell.

"""

# Step 1: Filter sentences that potentially contain U.S. states
relevant_sentences = extract_relevant_sentences(long_text)

# Step 2: Extract addresses from those sentences
extracted_addresses = extract_addresses_from_sentences(relevant_sentences)

print("Relevant Sentences:", relevant_sentences)
print("Extracted Addresses:", extracted_addresses)


Relevant Sentences: ["Dinner reservations are at 7 PM at 'The Orchard', a cozy restaurant nestled within the city's \nhistorical district at 456 Elm Street, Apt 101, New York, NY. \n", "As the curtain falls, our thoughts will already be on tomorrow's plans, which include a visit \nto the famous 789 Pine Avenue, Los Angeles, CA, for a guided tour of historic homes and gardens.\n\n"]
Extracted Addresses: ['Elm Street', 'Pine Avenue']


In [23]:

# looking for specific tag can be interesting approach

import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
doc = nlp("Will you go there?")
pattern = [{"IS_SENT_START": True, "TAG": "MD"}]
matcher = Matcher(nlp.vocab)
matcher.add("AUX_PATTERN" ,[pattern])
matches = matcher(doc)
print(len(matches))
mid, start, end = matches[0]
print(start, end, doc[start:end])
doc2 = nlp("I might go there.")
print(matcher(doc2))

1
0 1 Will
[]


In [24]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Define a pattern to match tokens that spaCy recognizes as numerical
pattern = [{"LIKE_NUM": True}]
matcher.add("NUM_PATTERN", [pattern])

# Example text with mixed numerical representations
text = "There are three apples for two dollars each. In total, that's six dollars."
doc = nlp(text)

# Find matches in the document
matches = matcher(doc)
for match_id, start, end in matches:
    print(start, end, doc[start:end])

# Testing on a more complex text
complex_text = "The distance between the cities is one hundred kilometers, roughly 62 miles."
doc2 = nlp(complex_text)
print(matcher(doc2))


2 3 three
5 6 two
14 15 six
[(8020647245559105862, 6, 7), (8020647245559105862, 7, 8), (8020647245559105862, 11, 12)]


In [25]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")  # Load a spaCy medium model

matcher = Matcher(nlp.vocab)

# Define a pattern that looks for an address starting with a number in word form
address_pattern = [
    {"LIKE_NUM": True},  # Number in word form
    {"IS_ALPHA": True, "OP": "+"},  # One or more alphabetical tokens for the street name
    {"LOWER": {"IN": ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]}},  # Street type
]

# Add the pattern to the matcher
matcher.add("ADDRESS_IN_WORDS", [address_pattern])

# Example text
text = "Let's meet at one eighty-two Pine Street next Saturday."

# Process the text with spaCy
doc = nlp(text)

# Find matches in the text
matches = matcher(doc)

# Extract and print the matched spans
for match_id, start, end in matches:
    span = doc[start:end]  # The matched span
    print(f"Matched address: {span.text}")


Matched address: two Pine Street


In [26]:
# 2 Syntax Support and Regex

# in operator
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)
doc = nlp("Good morning, I'm here. I'll say good evening!!")
pattern = [{"LOWER": "good"},
{"LOWER": {"IN": ["morning", "evening"]}},{"IS_PUNCT": True}]
matcher.add("greetings",  [pattern])
matches = matcher(doc)
for mid, start, end in matches:
  print(start, end, doc[start:end])

0 3 Good morning,
10 13 good evening!


In [27]:
import spacy
from spacy.matcher import Matcher

# length attribute
nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)
doc = nlp("I suffered from Trichotillomania when I was in college. The doctor prescribed me Psychosomatic medicine.")
pattern = [{"LENGTH": {">=" : 10}}]
matcher.add("longWords",  [pattern])
matches = matcher(doc)
for mid, start, end in matches:
  print(start, end, doc[start:end])

3 4 Trichotillomania
12 13 prescribed
14 15 Psychosomatic


In [29]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")  # Load a spaCy medium model

matcher = Matcher(nlp.vocab)

# Define a pattern that uses POS tags to improve address matching
address_pattern = [
    {"LIKE_NUM": True, "POS": "NUM"},  # Numeric word form, tagged as numeral
    {"POS": "PROPN", "OP": "+"},  # One or more proper nouns for the street name
    {"POS": "NOUN", "LOWER": {"IN": ["street", "avenue", "road", "boulevard", "lane", "drive"]}},  # Street type as a noun
]

# Add the pattern to the matcher
matcher.add("ADDRESS_IN_WORDS_POS", [address_pattern])

# Example text
text = "We're heading to twenty two Baker Street for the event."

# Process the text with spaCy
doc = nlp(text)

# Find matches in the text
matches = matcher(doc)

# Extract and print the matched spans
for match_id, start, end in matches:
    span = doc[start:end]  # The matched span
    print(f"Matched address: {span.text}")


In [30]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

doc = nlp("My name is Alice and his name was Elliot.")
pattern = [{"LOWER": "name"},{"LEMMA": "be"},{}]
matcher.add("pickName", [pattern])
for mid, start, end in matcher(doc):
  print(start, end, doc[start:end])

1 4 name is Alice
6 9 name was Elliot


In [32]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Define the pattern to extract street addresses written with numbers in words
address_pattern = [
    {"LIKE_NUM": True},  # Street number in word form
    {"POS": {"IN": ["PROPN", "NOUN"]}},  # Street name as a proper noun or noun
    {"LEMMA": {"IN": ["street", "avenue", "road", "boulevard", "lane", "drive"]}, "POS": "NOUN"}  # Street type
]

# Add the pattern to the matcher
matcher.add("STREET_ADDRESS_IN_WORDS", [address_pattern])

# Example text with addresses written in words
text = "They live on twenty two Baker Street. Another friend lives on thirty four Elm Avenue."

# Process the text with spaCy
doc = nlp(text)

# Apply the matcher to the processed document
matches = matcher(doc)
print(matches)
# Print the matched spans
for match_id, start, end in matches:
    print(f"Matched address: {doc[start:end]}")


[]


In [33]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Define patterns to capture numeric street numbers in word form, followed by a street name and type
# This approach uses a combination of POS tagging and pattern flexibility to improve match accuracy
address_pattern = [
    {"POS": "NUM", "OP": "+"},  # One or more numeric tokens, allowing for multi-word numbers
    {"IS_ALPHA": True, "OP": "+"},  # One or more tokens for the street name, may include proper nouns and common nouns
    {"LEMMA": {"IN": ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]}, "POS": "NOUN"}  # Specific street types as nouns
]

# Add the pattern to the matcher
matcher.add("STREET_ADDRESS", [address_pattern])

# Example text with addresses written in words
text = "Let's meet at twenty two Baker Street. Don't forget the party at one hundred First Avenue."

# Process the text with spaCy
doc = nlp(text)

# Apply the matcher to the processed document
matches = matcher(doc)

# Print the matched spans
for match_id, start, end in matches:
    print(f"Matched address: {doc[start:end]}")


In [34]:
# seems that funneling is better approach
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Adjusted pattern focusing on street types
address_pattern = [
    {"POS": "PROPN", "OP": "*"},  # Optional proper nouns for street names
    {"LEMMA": {"IN": ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]}, "POS": "NOUN"}  # Street type
]

matcher.add("STREET_ADDRESS", [address_pattern])

def extract_addresses(text):
    doc = nlp(text)
    matches = matcher(doc)
    addresses = []

    for match_id, start, end in matches:
        # Extend the match backwards to include number words
        num_start = start
        while num_start > 0 and doc[num_start - 1].like_num:
            num_start -= 1
        addresses.append(doc[num_start:end].text)

    return addresses

# Example text
text = "They moved to one hundred twenty-two Elm Street after living on forty-two Maple Avenue for years."

# Extract addresses
addresses = extract_addresses(text)
print("Extracted Addresses:", addresses)


Extracted Addresses: []


In [35]:
import spacy
from spacy.tokens import Span

nlp = spacy.load("en_core_web_md")

def extract_street_addresses(doc):
    addresses = []
    # Define street type keywords for pattern matching
    street_types = ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]
    # Iterate through tokens to find potential addresses based on street type
    for token in doc:
        if token.lemma_ in street_types:
            # Start from the token, move backwards to collect the street number and name
            start = token.i
            while start > 0 and (doc[start - 1].like_num or doc[start - 1].pos_ in ["PROPN", "NOUN"]):
                start -= 1
            # Create a span from the start to the current token's position + 1 (to include the street type token itself)
            span = Span(doc, start, token.i + 1, label="ADDRESS")
            addresses.append(span.text)
    return addresses

# Example text
text = "They first lived at twenty two Baker Street before moving to one hundred twenty-three Elm Avenue. Another place was at fifty First Avenue."
doc = nlp(text)

# Extract addresses
addresses = extract_street_addresses(doc)
print("Extracted Addresses:", addresses)


Extracted Addresses: []


In [36]:
import spacy
from spacy.tokens import Span, Token
import re

nlp = spacy.load("en_core_web_md")

# Custom function to check if a token or span starts with a number (in words)
def starts_with_number(text):
    number_words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten',
                    'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen',
                    'eighteen', 'nineteen', 'twenty']
    # Check if the first word of the text is in the number_words list
    first_word = text.split()[0].lower()
    return first_word in number_words

# Function to extract addresses with numbers spelled out
def extract_spelled_out_addresses(doc):
    addresses = []
    street_type_keywords = ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]

    for token in doc:
        # Check if the token is a street type
        if token.text.lower() in street_type_keywords:
            # Create a span for the potential address
            start = token.i - 2 if token.i >= 2 else 0
            end = token.i + 1
            address_span = doc[start:end]

            # Check if the span starts with a number (in words)
            if starts_with_number(address_span.text):
                addresses.append(address_span.text)

    return addresses

# Example text
text = "They first lived at twenty two Baker Street before moving to one hundred twenty three Elm Avenue. Another place was at fifty First Avenue."
doc = nlp(text)

# Extract addresses
addresses = extract_spelled_out_addresses(doc)
print("Extracted Addresses:", addresses)


Extracted Addresses: ['two Baker Street', 'three Elm Avenue']


In [37]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Pattern to capture commonly mistyped numeric values in the context of an address
# For simplicity, this example focuses on mistyped numbers 1-9, but you can expand this
pattern = [
    {"TEXT": {"REGEX": "^(one|two|three|four|five|six|seven|eight|nine|1|2|3|4|5|6|7|8|9|onw|tow|thre|foru|fiv|sxi|sven|egiht|nien)$"}},
    {"IS_ALPHA": True, "OP": "+"},  # Street name
    {"LOWER": {"IN": ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "lane", "ln", "drive", "dr"]}},  # Street type
]

matcher.add("MISTYPED_NUM_STREET", [pattern])

# Example text with a mistyped numeric value
text = "Let's meet at onw Main Street."

# Process the text with spaCy
doc = nlp(text)

# Apply the matcher to the processed document
matches = matcher(doc)

# Print the matched spans
for match_id, start, end in matches:
    print(f"Mistyped Address: {doc[start:end]}")


Mistyped Address: onw Main Street


In [38]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")
matcher = Matcher(nlp.vocab)

# Example pattern to capture street types and possibly the street name
# Note: Direct regex use is limited to the TEXT field; other attributes are used for structural matching
patterns = [
    {"IS_DIGIT": True, "OP": "?"},  # Optional digit (street number)
    {"POS": "PROPN", "OP": "+"},  # One or more proper nouns (street name)
    {"TEXT": {"REGEX": "^(Street|St\.|Avenue|Ave\.|Road|Rd\.|Boulevard|Blvd\.)$"}, "OP": "?"},  # Street type with regex
]

matcher.add("STREET_ADDRESS", [patterns])

# Example text
text = "They live at 123 Elm Street. Another place is on Pine Ave. Let's meet at the corner of 4th and Main."

doc = nlp(text)

matches = matcher(doc)

for match_id, start, end in matches:
    span = doc[start:end]
    print("Matched address:", span.text)


Matched address: 123 Elm
Matched address: 123 Elm Street
Matched address: Elm
Matched address: Elm Street
Matched address: Street
Matched address: Pine
Matched address: Ave
Matched address: Pine Ave
Matched address: Main


In [40]:
# 03 PhraseMatcher and Entity Ruler

# matching long dictionaries

import spacy
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab)

terms = ["Angela Merkel", "Donald Trump"]
patterns = [nlp.make_doc(term) for term in terms]

matcher.add("politiciansList", None, *patterns)

doc = nlp("3 EU leaders met in Berlin. German chancellor Angela Merkel first welcomed the US president Donald Trump. The following day Alexis Tsipras joined them in Brandenburg.")

matches = matcher(doc)

for mid, start, end in matches:
    print(start, end, doc[start:end])

9 11 Angela Merkel
16 18 Donald Trump


In [41]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab)

# Common street types as terms
street_types = ["Street", "Avenue", "Road", "Boulevard", "Lane", "Drive", "Terrace", "Place", "Square", "Circle"]
# Create patterns from the street types
patterns = [nlp.make_doc(street_type) for street_type in street_types]
matcher.add("STREET_TYPES", None, *patterns)

# Example text with several street names and types
text = "I've lived on Elm Street and Pine Avenue. My friend moved to Birch Road. We plan to visit Maple Boulevard next week."

doc = nlp(text)

matches = matcher(doc)

for mid, start, end in matches:
    # Extend the match to potentially include the street number or name
    # This example simply prints the match, but you could expand this logic to include nearby tokens
    print("Matched street type:", doc[start:end])


Matched street type: Street
Matched street type: Avenue
Matched street type: Road
Matched street type: Boulevard


In [42]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
terms = ["Asset", "Investment", "Derivatives", "Demand",  "Market"]
patterns = [nlp.make_doc(term) for term in terms]
matcher.add("financeTerms", patterns)
doc = nlp("During the last decade, derivatives market became an asset class of their own and influenced the financial landscape strongly.")
matches = matcher(doc)
for mid, start, end in matches:
  print(start, end, doc[start:end])

5 6 derivatives
6 7 market
9 10 asset


In [43]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

# List of common street types
street_types = ["street", "ave", "road", "boulevard", "lane", "drive", "terrace", "place", "square", "circle"]
patterns = [nlp.make_doc(street_type) for street_type in street_types]
matcher.add("STREET_TYPES", patterns)

text = "I used to live on Maple Avenue but recently moved to Pine Road. Let's meet at the corner of 5th Avenue and 42nd Street."
doc = nlp(text)

matches = matcher(doc)

for mid, start, end in matches:
    # Assuming the street name precedes the street type, look 1-3 tokens before the match
    # Adjust the range as necessary based on your data
    match_start = max(start - 3, 0) # magic number previous 3 words !!!
    match_end = end  # The end of the match is the end of the street type
    print("Potential address:", doc[match_start:match_end])


Potential address: moved to Pine Road
Potential address: Avenue and 42nd Street


In [44]:
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_md")
matcher = PhraseMatcher(nlp.vocab, attr="SHAPE")
ip_nums = ["127.0.0.0", "127.256.0.0"]
patterns = [nlp.make_doc(ip) for ip in ip_nums]
matcher.add("IPNums", patterns)
doc = nlp("This log contains the following IP addresses: 192.1.1.1 and 192.12.1.1 and 192.160.1.1 .")
for mid, start, end in matcher(doc):
  print(start, end, doc[start:end])

8 9 192.1.1.1
12 13 192.160.1.1


In [56]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_md")

matcher = Matcher(nlp.vocab)
pattern = [{"ENT_TYPE": "PERSON"}]
matcher.add("personEnt",  [pattern])
doc = nlp("Bill Gates visited Berlin.")
matches = matcher(doc)
for mid, start, end in matches:
  print(start, end, doc[start:end])

0 1 Bill
1 2 Gates


In [57]:
doc = nlp("I have an acccount with chime since 2017")
print(doc.ents)
patterns = [{"label": "ORG", "pattern": [{"LOWER": "chime"}]}]
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patterns)
print(doc.ents)
print(doc[5].ent_type_)

(2017,)
(2017,)

